In [1]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import pickle, random
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.cross_validation import StratifiedShuffleSplit
feature_folder = "/home/neo/ml1/code/ML1_project/features/"
random.RandomState=0

/home/neo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
LOG_DIR = '/home/neo/ml1/code/ML1_project/rbm_log_4096'
metadata = os.path.join(LOG_DIR,'metadata.tsv')
labels = pickle.load(open(feature_folder+"labels.pkl", "rb"))
features = np.array(pickle.load(open(feature_folder+"features_rbm_4096.pkl", "rb")))
words = pickle.load(open(feature_folder+"final_words.pkl","rb"))
features = [np.ndarray.tolist(element) for element in features] 

In [3]:
new_features=[]
new_labels=[]
per_domain_samples = 10000//7
new_features = random.sample(features[:15404],per_domain_samples)
new_features += random.sample(features[15405:25836],per_domain_samples)
new_features += random.sample(features[25837:28607],per_domain_samples)
new_features += random.sample(features[28608:41803],per_domain_samples)
new_features += random.sample(features[41804:61082],per_domain_samples)
new_features += random.sample(features[61083:87000],per_domain_samples)
new_features += random.sample(features[87001:168926],per_domain_samples)

new_labels = random.sample(labels[:15404],per_domain_samples)
new_labels += random.sample(labels[15405:25836],per_domain_samples)
new_labels += random.sample(labels[25837:28607],per_domain_samples)
new_labels += random.sample(labels[28608:41803],per_domain_samples)
new_labels += random.sample(labels[41804:61082],per_domain_samples)
new_labels += random.sample(labels[61083:87000],per_domain_samples)
new_labels += random.sample(labels[87001:168926],per_domain_samples)

In [4]:
np.array(features[0])-np.array(features[1])

array([ 0.0361582 , -0.03402048, -0.02112159, ..., -0.00178939,
        0.02064374,  0.02015078])

In [5]:
len(new_labels)

9996

In [6]:
print("Length of labels:{0}\nLength of features:{1}\nLength of words:{2}\n"\
      .format(len(new_labels),len(new_features),len(words)))

Length of labels:9996
Length of features:9996
Length of words:11764



In [7]:
# labels = [label[0] for label in labels]
new_labels2 = []
for label in labels:
    new_labels2.append(new_labels[0])

In [8]:
# split = min(labels.count("cooking"),labels.count("robotics"))
# strat = ["cooking" for x in range(split//2)] + ["robotics" for x in range(split//2)] 

In [9]:
# stratSplit = StratifiedShuffleSplit(labels, 1, test_size=0.77,random_state=42)
# StratifiedShuffleSplit(y, n_iter=1, test_size=0.5)

In [10]:
feature_train, feature_test, label_train, label_test = train_test_split(new_features, new_labels, \
                                                    test_size=0.5, random_state=0)
# word_train, word_test, label_train, label_test = train_test_split(words, new_labels, \
#                                                     test_size=0.5, random_state=0)

In [11]:
type(feature_train[0])

list

In [12]:
# mnist = input_data.read_data_sets('MNIST_data')
X_init = tf.placeholder(tf.float32, shape=(len(feature_train), len(features[0])))
images = tf.Variable(X_init)
# images = tf.Variable(feature_train, name='features')
# images = tf.Variable(X_test, name='images')
# im_labels = []
# for word in y_test:
#     im_labels.append(labels[words.index(word)])

In [13]:
with open(metadata, 'w') as metadata_file:
    for row in label_train:
        metadata_file.write('%s\n' % row)

In [14]:
with tf.Session() as sess:
    saver = tf.train.Saver([images])

    sess.run(tf.initialize_all_variables(), feed_dict={X_init: feature_train})
    saver.save(sess, os.path.join(LOG_DIR, 'features.ckpt'))

    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = images.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = metadata
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
